In [1]:
import numpy as np
import psycopg2
import datetime
import math

In [2]:
npyname = "kddi_00_new_result"
sttime = "2022-04-01 00:00:00"

In [5]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [6]:
#道路情報に重なるメッシュIDをすべて取得
sql = "SELECT osm_id FROM roadmap"
cur =connection.cursor()
cur.execute(sql)
keylist = []
for row in cur:
    sql = "SELECT a.grid_code"
    sql += " FROM pointdata a , roadmap r "
    sql += " WHERE ST_Distance(a.geom, r.geom) < 0.003 "
    sql += " AND r.osm_id = " + str(row[0])

    cur2 =connection.cursor()
    cur2.execute(sql)
    childlist = []
    for row2 in cur2:
        childlist.append(row2[0])
    cur2.close()
    keylist.append([str(row[0]), childlist])
cur.close()

In [7]:
#道路ID, メッシュID
keylist

[['27755814',
  ['50324595243',
   '50324595244',
   '50324595241',
   '50324595242',
   '50324596131',
   '50324596132',
   '50324595214',
   '50324595223',
   '50324595224',
   '50324596113',
   '50324596114',
   '50324595212',
   '50324595221',
   '50324595222',
   '50324596111',
   '50324596112',
   '50324596121',
   '50324585443',
   '50324585444',
   '50324586333',
   '50324586334',
   '50324586343',
   '50324585441',
   '50324585442',
   '50324586331',
   '50324586332',
   '50324586341',
   '50324585424',
   '50324586313',
   '50324586314',
   '50324586323',
   '50324585422',
   '50324586311',
   '50324586312',
   '50324586133',
   '50324586134']],
 ['27755914',
  ['50324525414',
   '50324525423',
   '50324525411',
   '50324525412',
   '50324525421',
   '50324525422',
   '50324525233',
   '50324525234',
   '50324525243',
   '50324525244']],
 ['27755921', []],
 ['27755953', []],
 ['27808255',
  ['50326507413',
   '50326507414',
   '50326507423',
   '50326507424',
   '50326507411'

In [8]:
len(keylist)

3715

In [9]:
f = open('list.txt', 'w')
for x in keylist:
    f.write(str(x) + "\n")
f.close()

In [ ]:
#データが存在する道路IDごとに交通量を平均化して取得する
cntDt = 0
wks = ""
wknp=""
find = 0
dtstart = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')

cnts = 0
for row in keylist:
    if cnts % 100 == 0:
        print("cnt="+str(cnts))
    cnts = cnts + 1
    if wks == "":
        wks = row[0]
    elif wks != row[0]:
        if cntDt != 0:
            if find != 0:
                a_1 = np.array([[wks, np.array(lstItems)]])
                wknp = np.append(wknp, a_1, axis=0)
            else:
                find = 1
                a_1 = np.array([[wks, np.array(lstItems)]])
                wknp = a_1
        wks = row[0]
        cntDt = 0

    lstItems = [-1, -1, -1, -1,  -1,-1, -1, -1,  -1, -1,-1, -1,   -1,-1, -1,-1,   -1,-1, -1, -1 ,  -1, -1, -1, -1]

    print(row[0])
    for meshid in row[1]:
        
        sql = "SELECT times,tbran from " + npyname + " WHERE mesh = " + meshid + " and times < '2022-04-02 00:00:00'  order by times"
        cur =connection.cursor()
        cur.execute(sql)
        finds = 0
        
        for row2 in cur:
            dtnow = datetime.datetime.strptime(str(row2[0]), '%Y-%m-%d %H:%M:%S')
            dtdiff = dtnow - dtstart
            finds = math.floor(dtdiff.seconds / 60 / 15)
            if lstItems[finds] == -1:
                lstItems[finds] = row2[1]
            else:
                lstItems[finds] = (lstItems[finds] + row2[1]) / 2
            cntDt = cntDt + 1
        cur.close()
if cntDt != 0:
    if find != 0:
        a_1 = np.array([[wks, np.array(lstItems)]])
        wknp = np.append(wknp, a_1, axis=0)
    else:
        a_1 = np.array([[wks, np.array(lstItems)]])
        wknp = a_1
print("complete.")


In [8]:
connection.close()

In [ ]:
wknp

In [ ]:
np.savetxt(npyname + '_loaddata3.csv',wknp,fmt="%s", delimiter='\t')

In [70]:
sql = "CREATE TABLE " + npyname + "_loadsummary3 (times TIMESTAMP NULL DEFAULT NULL,osm_id BIGINT NULL DEFAULT NULL,tbran DOUBLE PRECISION NULL DEFAULT NULL )"
cur =connection.cursor()
cur.execute(sql)
cur.close()
connection.commit()


DuplicateTable: リレーション"kddi_12_param_new_result_loadsummary3"はすでに存在します


In [ ]:
cur =connection.cursor()

for items in wknp:
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    for params in items[1]:
        sql = "INSERT INTO " + npyname + "_loadsummary3 (times, osm_id,tbran) VALUES ('"+ str(dte) + "',"  + str(items[0]) + "," + str(params) + ");"
        cur.execute(sql)
        dte = dte + datetime.timedelta(minutes=15)
cur.close()
connection.commit()

In [91]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

cur =connection.cursor()

with open(npyname + '_loaddata3.csv') as f:
    lines=f.readlines()
    keys = ""
    for line in lines:
        myarray = np.fromstring(line, dtype=float, sep='\t')
        for i in range(len(myarray)):
            if myarray[i] > 10000:
                keys = str(myarray[i])
                dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
                linesplit = line.split('\t')
                resultdt = line.replace(str(linesplit[0]) + '\t', '')
                resultdt = resultdt.replace('\n]', '')
                resultdt = resultdt.replace('[  ', '')
                resultdt = resultdt.replace('[ ', '')
                resultdt = resultdt.replace('  ', ' ')
                resultdt = resultdt.replace('  ', ' ')
                resultdt = resultdt.replace('. ', '')
                resultdt = resultdt.replace('  ', ' ')
                resultdt = resultdt.replace('[', '')
                #print(resultdt)
                myresult = resultdt.split(' ')
                for j in range(len(myresult)):
                    sql = "INSERT INTO " + npyname + "_loadsummary3 (times, osm_id,tbran) VALUES ('"+ str(dte) + "',"  + keys + "," + str(myresult[j]) + ");"
                    #print(sql)
                    cur.execute(sql)                    
                    dte = dte + datetime.timedelta(minutes=15)
            else:
                sql = "INSERT INTO " + npyname + "_loadsummary3 (times, osm_id,tbran) VALUES ('"+ str(dte) + "',"  + keys + "," + str(myarray[i]) + ");"
                #print(sql)
                cur.execute(sql)
                dte = dte + datetime.timedelta(minutes=15)
cur.close()
connection.commit()

C:\Users\BIGDATA_DEV\AppData\Local\Temp\ipykernel_7568\2697937138.py:9: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  myarray = np.fromstring(line, dtype=float, sep='\t')


0.72071951 0.88694262 0.37089795 0.54015305 0.56549025 0.97066908

0.92542148 1.16430501 0.70436894 0.66731327 0.71364717 0.47027655

8.71160555 7.23971536 10.88203349 16.56183012 16.62151797 21.19031222

3.18331256 3.19951632 2.94271282 3.91521453 3.9254155 4.54364727

0.35628168 1.04015671 1.41832554 2.32676296 2.39409554 4.17116643

2.10627277 1.93995674 2.94888617 3.24285382 3.39724093 3.81485019

1.04099248 1.31017178 1.12660355 1.51411205 1.50981441 2.51537515

1.42541566 1.95971096 5.12207692 6.98429793 7.19155029 16.36061877

2.53350007 2.62175279 1.27523023 3.06850132 2.99343272 4.87822643

2.02708533 3.00022607 3.5505783 4.79903467 4.84330468 7.75087447

23.27798689 29.29204618 34.82813046 43.35187011 44.203164

0.50915247 0.73760724 0.47166333 0.6554995 0.66883616 0.95139558

1.43266312 1.24275007 1.26646064 1.77173241 1.7937114 1.48808097

7.80450483 11.62209204 11.98665795 33.17002405 32.86247525

1.82205036 1.89608559 1.28638457 1.25146083 1.2338585 2.0905147

0.30429155 

5.94304663 7.5447695 11.08845438 17.37171077 17.31094146

2.19873561 3.27694687 3.11493301 3.85001957 3.9257158 5.37070885

3.9937736 3.5890867 3.98510819 4.35352099 4.43639198 3.64118336

6.32113035 7.69102916 5.10580789 5.67420794 5.97291072 6.36639408

2.46085448 2.90564668 2.5448911 2.83033837 2.87516403

3.66337234 4.29069824 6.26531802 7.70920672 7.68275229

7.67375088 9.14512079 7.80120745 9.71527293 9.74757973 14.16614734

1.30553319 1.45423309 2.12947345 2.41186165 2.44035956 2.91528087

17.80531219 25.10048828 35.29496529 49.18039326 49.02931443

34.81558316 35.28965971 51.42811202 60.3807064 60.84594623

7.36607018 8.90449566 10.24331622 13.1700034 13.24352773

8.54728181 8.07263855 10.44420488 12.286906 12.35433363 16.24689927

35.43518365 41.48763388 38.31840759 42.41250221 42.85288571

6.85244386 8.28962923 7.57083634 9.29877004 9.35989429

2.02859971 2.99977315 4.18387555 5.61853488 5.7214823 8.97977298

68.32728909 63.54733235 90.42480026 99.13533829 101.70511751

4.747

0.42556984 0.39734096 0.83401958 0.91885609 0.98070759 2.02836831

3.51625966 5.01378402 3.01515401 3.66521033 3.75434745 5.1694899

0.37888475 0.36192489 0.21755678 0.26952059 0.27775973 0.90182913

0.86147159 1.21682381 3.60914266 5.04848354 5.06529372 8.55541517

0.55208129 0.46063778 0.59687788 1.14089073 1.13375409 2.45137974

0.36867407 0.33751919 0.68651421 1.39748711 1.40257966 2.88307062

0.72234875 0.82835851 1.18145787 1.48050735 1.52435721 2.10360211

0.4048282 0.35932492 0.74801201 0.89029829 0.92910889 2.48217492

1.10978449 1.20164662 1.44946238 1.67598341 1.62157709 2.12865558

0.37888475 0.36192489 0.21755678 0.26952059 0.27775973 0.90182913

1.10595943 1.07317723 2.33451558 3.04340201 3.05419945 4.64010055

1.61935804 2.19469686 1.05078954 2.46158594 2.52755549 3.40674249

0.30658444 0.35049899 0.35379691 0.86963465 0.90342562 1.45044876

0.31318969 0.63488113 1.63746569 2.47323618 2.48851136 3.55408941

0.92374299 1.67068503 0.48142218 0.87719546 0.89507347 1.0469033

9.06098536 12.5024802 15.76338773 20.81314212 21.02015607

46.95575535 74.78400592 122.51488685 177.2070082 175.89417113

7.24511469 10.06959107 9.59839812 12.38252014 12.62511993

3.18403711 4.55994066 6.28804094 9.32137435 9.40782013

2.37715533 3.0581936 1.94330868 2.34551812 2.56145478 2.99035134

7.0716305 10.27685565 12.48452976 17.05509197 16.98485068

7.66800735 9.63714085 15.27841413 20.30823119 20.44562959

17.45499546 21.33017782 28.39033136 35.1361486 35.56109081

3.69547279 4.63762019 8.09105192 10.79201398 10.86741263

15.13322381 19.41938593 25.96390098 33.64410103 34.57722964

24.57418707 38.09240409 63.8665517 91.69604328 91.12114152

4.54902749 4.54454793 7.8914366 8.46682258 8.73352286 9.12032949

6.98597588 8.45933625 7.65003833 9.43498653 9.50655029

5.50716879 5.30017253 9.64024996 10.24564012 10.51673894

19.41416663 40.34241135 26.77325982 34.47652575 35.5648718

10.24315773 10.87891221 12.53151245 14.33804161 14.41247579 17.31865837

1.42645933 3.92863116 5.592

...

4.11570581 4.27007981 3.75407363 3.63579436 3.78790247 6.77699196

0.81443495 0.51697073 0.39473164 0.50200355 0.49786187 0.76686823

0.61481178 0.31356071 1.06613808 1.27832865 1.3013683 3.84087951

4.57727993 5.44508773 6.10286497 7.25788151 7.3087816 9.9087534

4.78325207 4.59635284 7.21484548 8.2830199 8.5364078 9.91262067

0.43590535 0.38543833 0.74486093 0.94578066 0.9835181 1.29445461

3.43940897 3.71688786 3.50707143 3.16634635 3.2656337 4.88788652

0.96276125 2.04805237 1.25091513 2.42276667 2.48339156 2.50929542

4.6695291 6.58274891 3.90508545 7.0103406 6.8862765 11.48940838

0.84245983 0.96411483 1.98233805 2.17384521 2.2121927 2.79553596

5.02052534 4.67030154 9.68563159 11.66099496 12.01461932 15.37849738

1.46368495 1.97565487 1.42601602 1.66426025 1.65473214 3.49315055

1.35078657 1.57216849 3.77477038 4.72892905 5.05998299 7.93444148

1.5626374 2.20219344 1.50369584 1.8725724 1.92844265 3.01905455

9.17051086 10.50271553 8.6870008 10.40182966 10.34743337

0.45856112 0.

1.82203829 1.48201275 1.32120395 1.52800033 1.65216249 1.62745265

0.92705649 0.6729417 0.25675382 0.52198438 0.52824468 1.0624318

0.18252599 0.33485472 0.18848298 0.16312392 0.14844094 0.88994987

2.19399714 2.65252059 1.3245727 1.46876263 1.59377607 1.52812235

0.55826491 1.63188662 1.87495049 2.41114341 2.55506925 3.83934066

1.21422123 1.39690478 2.24094848 3.66244449 3.61337082 6.14661951

0.32151059 0.21969533 0.43864355 0.5312204 0.57114715 1.00260372

0.85040086 1.20630017 1.58915252 2.63265328 2.65483344 4.16030425

0.80452957 1.21997609 0.99251412 2.46415368 1.24154575 4.01339371

1.49550998 2.42140157 4.31912923 6.29726388 6.17292437 11.03601982

1.93543689 2.70833204 2.57177413 3.39319315 3.32897296 5.34926418

1.3973318 1.96959772 6.76888949 8.48611431 8.40818445

1.29025383 1.74291085 6.9451812 8.78904702 8.71183492

5.30745256 6.86501881 11.05087703 14.37378031 14.4857872

5.30752261 6.86518551 11.05091385 14.37379513 14.48579423

1.49869486 2.42847013 4.31827276 6.2941

2.10194327 1.94557228 1.9442761 4.54422015 4.59953197 6.11428184

1.21620919 1.58709622 0.65171104 0.96551233 0.92452798 1.40719

3.32511 1.98123161 4.9769136 6.95334221 6.51778043 10.49295142

5.25109002 5.13474809 7.31093701 10.53341515 10.69424681

1.3473224 1.7836365 0.90770733 1.69873768 1.6611927 2.66111609

1.28663865 1.89633842 0.7580962 0.96748901 0.97453911 1.39528934

2.51348836 3.05750139 2.8017238 3.28038667 3.35933918 4.42045381

0.57859294 0.69600665 0.5179804 0.67481017 0.70764084 2.66754345

0.69752849 1.26539344 0.89495316 1.27776949 1.28267029 2.14207887

0.3205864 0.74016745 1.65151478 2.6345315 2.62400187 5.13743695

2.08693858 2.6010528 1.6319109 1.59864081 1.57606187 3.56074447

1.0439117 0.73669262 2.51224475 2.9688134 3.05004597 5.37038445

1.28283656 1.91233683 1.43426896 1.76130997 1.81338157 2.56324484

0.18619423 0.61182861 0.78786334 1.2252537 1.23075419 2.41995703

2.69772868 3.55425062 2.84080278 4.90338301 4.79954965 7.90035254

0.41146728 1.59292845 1.

6.74351246 7.12101338 1.33623712 1.12979522 1.23258152 0.63736888

1.63618604 2.61178415 6.16228301 15.6954543 15.89004896

1.622355 2.94877553 5.40515075 13.22333201 13.29113314

5.26843622 8.77966468 5.5842771 7.65407902 7.83275311

3.87294601 5.90965831 3.87854664 4.39585627 4.50044128 6.5057008

3.74598622 5.61845285 3.77215522 4.28274301 4.37091435 7.78400267

3.44993321 4.91592081 2.69684323 3.6945282 3.70117381 6.51118268

7.39759183 8.56410594 6.66048888 7.9837922 8.13936833 11.63477336

11.38577379 12.65517281 11.19559373 24.82871733 25.38011525

8.19221721 7.55322339 10.20449787 12.70211939 12.96019604

2.89729572 3.03102845 4.27059125 6.98204573 7.17396684 8.96561665

3.2494545 3.41305556 4.16839093 6.58935691 6.73868047 8.51461304

3.48106909 4.12098344 5.92130608 7.52367749 7.30668256 11.98411908

6.76608018 9.43994714 9.19145372 13.42460837 13.17109791

7.05266775 8.31906269 11.06437048 15.19809434 15.13827437

15.86345163 19.71828175 23.19081402 29.90883357 29.74740865



4.99229616 5.64911157 10.03514476 13.55347623 13.56512034

26.7455442 61.7139191 31.55851254 72.7025527 71.63800464

0.2794065 0.35896877 0.26104444 0.26504338 0.28921324 0.63161871

2.54588614 4.51070984 14.42607543 20.35604463 21.09669421

1.50071286 1.65601369 0.52791533 1.08871532 1.08265184 1.66899015

5.36570826 5.6381725 15.95277735 24.65099279 25.45552258

1.53561756 1.00333712 1.34811938 2.02177748 1.98410386 3.84712488

1.8504456 1.7212221 4.21921679 5.39392899 5.48540217 8.07702189

2.2100648 2.46788276 4.60849849 5.33613154 5.50422444 8.03866231

3.59913694 3.64440283 7.01344747 8.67833117 8.89351821

2.61911388 2.51619078 2.49671268 3.23241196 3.2199949 3.67639994

0.54192124 0.74426419 0.98068238 1.26866119 1.2694422 1.65654734

1.19738447 1.21531028 0.74568177 0.84809057 0.88245347 2.20783329

1.18994739 1.20962564 0.7437917 0.84366905 0.87801379 2.19600689

2.04495653 4.93505513 4.79156477 6.169212 6.24900567 9.43054894

14.47858786 17.84568227 19.78376356 23.67279483 2

In [ ]:
#CreateGeojsonFile